# pre processing

In [1]:
import pandas as pd

# read log data
with open('../raw data/training-log-corpus.log', 'r') as f:
    logs = f.readlines()

last_time = 0.0
last_student = ''
action_count = 0
l = []
for i, line in enumerate(logs):
    tokens = line.split('\n')[0]
    tokens = tokens.split('|')

    student_id = tokens[0]
    if len(student_id) != 8:
        print(
            f"ERROR: in line {i}, student id length should be 8, found {len(student_id)}!"
        )
        exit(0)

    action = tokens[1]

    # starting entries do not have time
    try:
        float(tokens[2])
    except:
        tokens.insert(2, 0.0)
    time = float(tokens[2])

    if last_student != student_id:
        last_time = 0.0
        print(f"{action_count} actions found for student {last_student}")
        action_count = 0

    # check is time is almost monotonic
    if last_time - time >= 1:
        print(
            f"ERROR: in line {i}, time is not monotonic! current action {action}. total action found {action_count}. student {student_id}"
        )

    last_time = time
    last_student = student_id
    action_count += 1

    try:
        detail = tokens[3]
    except:
        detail = ""

    others = ("|").join(tokens[4:])

    l.append({
        'episode': student_id,
        'action': action,
        'time': time,
        'detail': detail,
        'others': others
    })

df = pd.DataFrame(l,
                  columns=['episode', 'action', 'time', 'detail', 'others'])


# for students with "RESET"
# we are keeping if student reseted after planing signifcant part of the game (>70%)
# or resetted very early (<30%)
print("-- removing reset issues --")
dlt_idx = []
student_with_reset = df.loc[df['action'] == 'RESET']['episode'].unique()
for student in student_with_reset:
    df_s = df.loc[df['episode'] == student]
    total_steps = len(df_s)
    i = df.loc[(df['episode'] == student)
               & (df['action'] == 'RESET')].index[0]
    before_reset_steps = len(df_s.loc[:i])
    percent_before_reset = round(before_reset_steps / total_steps * 100, 2)
    drop_curr = []
    if percent_before_reset < 30.0:
        drop_curr = list(df_s.loc[df_s.index[0]:i + 1].index)
    elif percent_before_reset > 70.0:
        drop_curr = list(df_s.loc[i:].index)
    else:
        drop_curr = list(df_s.index)

    print(
        "student: {0} | total_steps: {1} | reset_at: {2}({3}%) | dropping: {4}"
        .format(student, total_steps, before_reset_steps, percent_before_reset,
                len(drop_curr)))
    dlt_idx += drop_curr
    

df = df.drop(index=dlt_idx).reset_index(drop=True)
df

0 actions found for student 
908 actions found for student 100-0001
737 actions found for student 100-0003
605 actions found for student 100-0004
851 actions found for student 100-0005
705 actions found for student 100-0006
708 actions found for student 100-0007
816 actions found for student 100-0008
624 actions found for student 100-0009
1183 actions found for student 100-0010
1132 actions found for student 100-0011
799 actions found for student 100-0012
854 actions found for student 100-0013
836 actions found for student 100-0014
631 actions found for student 100-0016
1291 actions found for student 100-0017
585 actions found for student 100-0018
873 actions found for student 100-0019
897 actions found for student 100-0020
683 actions found for student 100-0021
790 actions found for student 100-0022
596 actions found for student 100-0024
575 actions found for student 100-0025
883 actions found for student 100-0026
955 actions found for student 100-0027
883 actions found for student 10

,episode,action,time,detail,others
0,100-0001,STUDY,0.000,AnonymousAMiddleSchool-December2010t500,SelfReport-true
1,100-0001,SOLUTION,0.000,disease-Influenza,object-Egg
2,100-0001,ADAPTATION,1.170,select-test-count,random|selected-1
3,100-0001,AUTHENTICATE,42.915,press-authenticate,100-0001
4,100-0001,PDAOPEN,116.775,mainscreen,
...,...,...,...,...,...
329916,100-0828,DIALOG,3334.410,NPC-Utterance,kim|Comebackwhenyouhaveadiagnosis!
329917,100-0828,DIALOG,3338.070,menu-choice,3337.186523|3338.079346|0.892822|choice-6|Bye.
329918,100-0828,GAMEOVER,3338.610,,
329919,100-0828,ADAPTATION,3338.610,select-reflection-prompt,random|selected-1


In [2]:
from\
    collections import defaultdict

# creating action to feature maps
action_map = {
    'DIALOG': 's_dialog_turn',
    'OPEN': 's_open_door',
    'LOOKSTART': 's_view_poster',
    'PICKUP': 's_pickup_obj',
    'DROP': 's_drop_obj',
    'TESTOBJECT': 's_test_obj',
    'WORKSHEET': 's_worksheet',
    'QUIZ': 's_quiz',
    'PDAOPEN': 's_use_pda',
    'LABELING': 's_label_slide',
    'NOTES': 's_take_note',
    'BOOKREAD': 's_read_book',
    'BRYCECOMPUTER': 's_bryce_computer',
    'GAMEOVER': 's_end_game'
}

# TALK action needs to be segmented further for narrative planner
action_talk_map = {
    'cur-action-talk-bryce': 's_talk_bryce',
    'cur-action-talk-teresa': 's_talk_teresa', 
    'cur-action-talk-ford': 's_talk_ford_quen_rob', 
    'cur-action-talk-quentin': 's_talk_ford_quen_rob', 
    'cur-action-talk-robert': 's_talk_ford_quen_rob', 
    
    'cur-action-talk-extraa': 's_talk_others',
    'cur-action-talk-extrab': 's_talk_others',
    'cur-action-talk-elise': 's_talk_others',
    'cur-action-talk-kim': 's_talk_others',
    
}

# all types of adaptions. Not using mystry quiz (as this has 8 choise) and off-task behavior (not enough sample)
adaption_map = {
    'select-bryce-symptoms-level': 's_aes_bryce_symptoms',
    'select-teresa-symptoms-level': 's_aes_teresa_symptoms',
    'select-present-quiz': 's_aes_knowledge_quiz',
    'select-worksheet-level': 's_aes_diagnosis_feedback',
}

# start index of action triggers 
narrative_trigger_map = {
    's_aes_trigger_bryce_symptoms': -1, # two actions [0, 1] 
    's_aes_trigger_teresa_symptoms': 1, # three actions [2, 3, 4]
    's_aes_trigger_knowledge_quiz': 4, # two actions [5, 6]
    's_aes_trigger_diagnosis_feedback': 6 # three actions [7, 8, 9],
}

narrative_trigger_map_for_env = {
    0: ['s_aes_bryce_symptoms', 1],
    1: ['s_aes_bryce_symptoms', 2],
    2: ['s_aes_teresa_symptoms', 1],
    3: ['s_aes_teresa_symptoms', 2],
    4: ['s_aes_teresa_symptoms', 3],
    5: ['s_aes_knowledge_quiz', 1],
    6: ['s_aes_knowledge_quiz', 2],
    7: ['s_aes_diagnosis_feedback', 1],
    8: ['s_aes_diagnosis_feedback', 2],
    9: ['s_aes_diagnosis_feedback', 3]
}

student_trigger_map = {}
act_num = 0
for state in set(action_map.values()):
    student_trigger_map[state] = act_num
    act_num += 1
for state in set(action_talk_map.values()):
    student_trigger_map[state] = act_num
    act_num += 1
student_trigger_map['s_worksheet_submitted'] = act_num

states_student = list(student_trigger_map.keys()) + list(adaption_map.values())
init_row_student = defaultdict()
for state in states_student:
    init_row_student[state] = 0 
    
states_narrative = list(student_trigger_map.keys()) + list(adaption_map.values()) + list(narrative_trigger_map.keys())
init_row_narrative = defaultdict()
for state in states_narrative:
    init_row_narrative[state] = 0 

print(student_trigger_map)
print(states_narrative)

{'s_use_pda': 0, 's_open_door': 1, 's_read_book': 2, 's_pickup_obj': 3, 's_end_game': 4, 's_label_slide': 5, 's_view_poster': 6, 's_worksheet': 7, 's_quiz': 8, 's_dialog_turn': 9, 's_drop_obj': 10, 's_test_obj': 11, 's_take_note': 12, 's_bryce_computer': 13, 's_talk_others': 14, 's_talk_ford_quen_rob': 15, 's_talk_bryce': 16, 's_talk_teresa': 17, 's_worksheet_submitted': 18}
['s_use_pda', 's_open_door', 's_read_book', 's_pickup_obj', 's_end_game', 's_label_slide', 's_view_poster', 's_worksheet', 's_quiz', 's_dialog_turn', 's_drop_obj', 's_test_obj', 's_take_note', 's_bryce_computer', 's_talk_others', 's_talk_ford_quen_rob', 's_talk_bryce', 's_talk_teresa', 's_worksheet_submitted', 's_aes_bryce_symptoms', 's_aes_teresa_symptoms', 's_aes_knowledge_quiz', 's_aes_diagnosis_feedback', 's_aes_trigger_bryce_symptoms', 's_aes_trigger_teresa_symptoms', 's_aes_trigger_knowledge_quiz', 's_aes_trigger_diagnosis_feedback']


- do we need SOLUTION for s_aes_mystry_solution?
- kim revel adaption points player towards quentin's revel
- removing students who did restart. maybe worthwhile to include them later 
- 21 students didn't do pretest. thus nlg none. removing them. maybe worthwhile to include them later 

In [3]:
from copy import deepcopy
import pdb
pd.set_option("display.max_columns", None)

skipping_action = set()
skipping_adaption = set()

all_inserts_student = []
all_inserts_narrative = []
for student_id, rows in df.groupby(['episode']):
    row_insert_student = deepcopy(init_row_student)
    row_insert_student['episode'] = student_id
    row_insert_student['step'] = 0
    row_insert_student['done'] = False
    
    # this part is for narrative df only
    latest_adaptive_trigger = ''
    step_narrative = 0
    
    for i, row in rows.iterrows():
        if row['action'] == 'ADAPTATION':
            if row['detail'] not in adaption_map.keys():
                skipping_adaption.add(row['detail'])
                continue

            for adapt_feature in adaption_map.values():
                row_insert_student[adapt_feature] = 0

            row_insert_student[adaption_map[row['detail']]] = int(row['others'][-1:])
            
            # this part is for narrative df only
            
            # no adaption (among the selected ones) can happen as first action. 
            # thus, there must be another state before 
            row_insert_narrative = deepcopy(all_inserts_student[-1])
            for adapt_trigger_feature in narrative_trigger_map.keys():
                row_insert_narrative[adapt_trigger_feature] = 0
            row_insert_narrative[latest_adaptive_trigger] = 1
#             row_insert_narrative['step'] = step_narrative
            
            # converting different AES actions into uniform actions
            row_insert_narrative['action'] = narrative_trigger_map[latest_adaptive_trigger] + int(row['others'][-1:])
            row_insert_narrative['action_name'] = latest_adaptive_trigger + '_' +str(int(row['others'][-1:]))
            step_narrative += 1
            all_inserts_narrative.append(deepcopy(row_insert_narrative))
            continue

        if row['action'] == 'TALK':
            char = row['others'].split('|')[0]
            if char not in action_talk_map.keys():
                print(
                    "-- ERROR TALK UNKNOWN | action: {0} | detail: {1} | others: {2} --"
                    .format(row['action'], row['detail'], row['others']))
                break
            row_insert_student[action_talk_map[char]] += 1
            action = action_talk_map[char]
        
        elif row['action'] == 'DIALOG':
            if row['detail'] == 'menu-choice':
                if row['others'].split('|')[-1] == 'IthinkIhaveadiagnosis':
                    row_insert_student['s_worksheet_submitted'] += 1
                    action = 's_worksheet_submitted'
                else:
                    row_insert_student[action_map[row['action']]] += 1
                    action = action_map[row['action']]
            else:
                continue

        elif row['action'] in action_map.keys():
            row_insert_student[action_map[row['action']]] += 1
            action = action_map[row['action']]
        else:
            skipping_action.add(row['action'])
            if row['action'] == 'RESET':
                print(
                    "-- ERROR RESET happened for student {0} --".format(student_id))
            continue

        row_insert_student['action'] = student_trigger_map[action]
        row_insert_student['action_name'] = action
        all_inserts_student.append(deepcopy(row_insert_student))
        row_insert_student['step'] += 1
        
        if action == 's_end_game':
            break
        
        
        # this part is for narrative df only
        if action == 's_talk_bryce':
            latest_adaptive_trigger = 's_aes_trigger_bryce_symptoms'
        elif action == 's_talk_teresa':
            latest_adaptive_trigger = 's_aes_trigger_teresa_symptoms'
        elif action == 's_talk_ford_quen_rob':
            latest_adaptive_trigger = 's_aes_trigger_knowledge_quiz'
        elif action == 's_worksheet_submitted':
            latest_adaptive_trigger = 's_aes_trigger_diagnosis_feedback'
    
    if all_inserts_student[-1]['action_name'] != 's_end_game':
        row_insert_student['s_eng_game'] = 1
        row_insert_student['step'] += 1
        row_insert_student['action'] = student_trigger_map['s_end_game']
        row_insert_student['action_name'] = 's_end_game'
        all_inserts_student.append(deepcopy(row_insert_student))
            
    all_inserts_student[-1]['done'] = True
    all_inserts_narrative[-1]['done'] = True

    print('-- finished student {0} --'.format(student_id))

df_student_data = pd.DataFrame(all_inserts_student,
                               columns=['episode', 'step'] + states_student +
                               ['action', 'action_name', 'done'])

df_narrative_data = pd.DataFrame(all_inserts_narrative,
                               columns=['episode', 'step'] + states_narrative +
                               ['action', 'action_name', 'done'])


print("Actions skipped", skipping_action)
print("Adaptions skipped", skipping_adaption)

-- finished student 100-0001 --


/var/folders/ny/jqjpd68x1rj42v1309g8krv80000gn/T/ipykernel_3832/2465612682.py:10: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for student_id, rows in df.groupby(['episode']):


-- finished student 100-0003 --
-- finished student 100-0004 --
-- finished student 100-0005 --
-- finished student 100-0006 --
-- finished student 100-0007 --
-- finished student 100-0008 --
-- finished student 100-0009 --
-- finished student 100-0010 --
-- finished student 100-0011 --
-- finished student 100-0012 --
-- finished student 100-0013 --
-- finished student 100-0014 --
-- finished student 100-0016 --
-- finished student 100-0017 --
-- finished student 100-0018 --
-- finished student 100-0019 --
-- finished student 100-0020 --
-- finished student 100-0021 --
-- finished student 100-0022 --
-- finished student 100-0024 --
-- finished student 100-0025 --
-- finished student 100-0026 --
-- finished student 100-0027 --
-- finished student 100-0028 --
-- finished student 100-0029 --
-- finished student 100-0030 --
-- finished student 100-0031 --
-- finished student 100-0032 --
-- finished student 100-0033 --
-- finished student 100-0034 --
-- finished student 100-0035 --
-- finis

In [4]:
# merging scores in the feature list

df_score = pd.read_csv('../raw data/training-survey-corpus.csv')
df_score = df_score[[
    'Student ID', 'Gender', 'Game-Playing Frequency', 'Content Pre Total',
    'Normalized Learning Gain'
]]
df_score = df_score.rename(
    columns={
        'Student ID': 'episode',
        'Gender': 's_static_gender',
        'Game-Playing Frequency': 's_static_game_freq',
        'Content Pre Total': 's_static_pretest',
        'Normalized Learning Gain': 'reward'
    })

# few reward are NONE!
df_score = df_score.loc[df_score['reward']!='None']
df_score['reward'] = pd.to_numeric(df_score['reward'])


# making sure we have rewards for all students
df_student_data = df_student_data.loc[df_student_data['episode'].isin(df_score['episode'].unique())]
df_student_data = df_student_data.reset_index(drop=True)

df_narrative_data = df_narrative_data.loc[df_narrative_data['episode'].isin(df_score['episode'].unique())]
df_narrative_data = df_narrative_data.reset_index(drop=True)

df_score = df_score.loc[df_score['episode'].isin(df_student_data['episode'].unique())]


# splitting reward based on median
mid_reward = df_score['reward'].describe()['50%']
df_score.loc[df_score['reward']<mid_reward, 'reward'] = -100
df_score.loc[df_score['reward']>=mid_reward, 'reward'] = 100


df_student_data = df_student_data.merge(df_score,
                                        on=['episode'],
                                        how='left')

df_narrative_data = df_narrative_data.merge(df_score,
                                        on=['episode'],
                                        how='left')

df_narrative_data.loc[df_narrative_data['done']==False, 'reward'] = 0
df_student_data.loc[df_student_data['done']==False, 'reward'] = 0

df_student_data.head(50)

,episode,step,s_use_pda,s_open_door,s_read_book,s_pickup_obj,s_end_game,s_label_slide,s_view_poster,s_worksheet,s_quiz,s_dialog_turn,s_drop_obj,s_test_obj,s_take_note,s_bryce_computer,s_talk_others,s_talk_ford_quen_rob,s_talk_bryce,s_talk_teresa,s_worksheet_submitted,s_aes_bryce_symptoms,s_aes_teresa_symptoms,s_aes_knowledge_quiz,s_aes_diagnosis_feedback,action,action_name,done,s_static_gender,s_static_game_freq,s_static_pretest,reward
0,100-0001,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,s_use_pda,False,2,1,3,0.0
1,100-0001,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,s_use_pda,False,2,1,3,0.0
2,100-0001,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,s_open_door,False,2,1,3,0.0
3,100-0001,3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,s_use_pda,False,2,1,3,0.0
4,100-0001,4,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,s_use_pda,False,2,1,3,0.0
5,100-0001,5,4,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,14,s_talk_others,False,2,1,3,0.0
6,100-0001,6,4,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,9,s_dialog_turn,False,2,1,3,0.0
7,100-0001,7,4,1,0,0,0,0,0,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,0,9,s_dialog_turn,False,2,1,3,0.0
8,100-0001,8,4,1,0,0,0,0,0,0,0,3,0,0,0,0,1,0,0,0,0,0,0,0,0,9,s_dialog_turn,False,2,1,3,0.0
9,100-0001,9,4,1,0,0,0,0,0,0,0,4,0,0,0,0,1,0,0,0,0,0,0,0,0,9,s_dialog_turn,False,2,1,3,0.0


In [5]:
import numpy as np
# creating states
df_student_data['state'] = df_student_data.apply(lambda x: np.array([
    # 0-18 position matching student action numbers
    x['s_view_poster'],
    x['s_label_slide'],
    x['s_read_book'],
    x['s_test_obj'],
    x['s_bryce_computer'],
    x['s_end_game'], # 5
    x['s_take_note'],
    x['s_drop_obj'],
    x['s_dialog_turn'],
    x['s_open_door'],
    x['s_worksheet'],
    x['s_use_pda'],
    x['s_quiz'],
    x['s_pickup_obj'],
    x['s_talk_ford_quen_rob'], # 14
    x['s_talk_bryce'], # 15
    x['s_talk_others'], # 16
    x['s_talk_teresa'], # 17
    x['s_worksheet_submitted'], # 18
    
    # 19-22 position
    x['s_aes_bryce_symptoms'],
    x['s_aes_teresa_symptoms'],
    x['s_aes_knowledge_quiz'],
    x['s_aes_diagnosis_feedback'],
    
    # 23-25 position
    x['s_static_gender'], 
    x['s_static_game_freq'], 
    x['s_static_pretest'], 
    # 26 position new
    x['step'],
]), axis=1)

# creating states
df_narrative_data['state'] = df_narrative_data.apply(lambda x: np.array([
    # 0-18 position matching student action numbers
    x['s_view_poster'],
    x['s_label_slide'],
    x['s_read_book'],
    x['s_test_obj'],
    x['s_bryce_computer'],
    x['s_end_game'],
    x['s_take_note'],
    x['s_drop_obj'],
    x['s_dialog_turn'],
    x['s_open_door'],
    x['s_worksheet'],
    x['s_use_pda'],
    x['s_quiz'],
    x['s_pickup_obj'],
    x['s_talk_ford_quen_rob'],
    x['s_talk_bryce'],
    x['s_talk_others'],
    x['s_talk_teresa'],
    x['s_worksheet_submitted'],
    
    # 19-22 position
    x['s_aes_bryce_symptoms'],
    x['s_aes_teresa_symptoms'],
    x['s_aes_knowledge_quiz'],
    x['s_aes_diagnosis_feedback'],
    
    # 23-25 position
    x['s_static_gender'], 
    x['s_static_game_freq'], 
    x['s_static_pretest'],
    # 26 position new added
    x['step'],
    
    # 27-30 position
    x['s_aes_trigger_bryce_symptoms'],
    x['s_aes_trigger_teresa_symptoms'],
    x['s_aes_trigger_knowledge_quiz'],
    x['s_aes_trigger_diagnosis_feedback']
]), axis=1)


df_student_data.to_pickle('../processed_data/student_trajectories.pkl')
df_narrative_data.to_pickle('../processed_data/narrative_trajectories.pkl')
df_score.to_pickle('../processed_data/scores.pkl')

# random Tried LSTM based outcome predictor

In [ ]:
import pandas as pd
import numpy as np

df_all = pd.read_pickle('../processed_data/student_trajectories.pkl')
state_dim = 8
seq_len = 700

all_states = []
label = []

for student, df in df_all.groupby('student_id'): 
    states = df.apply(lambda x: np.append(x['state'][-8:-1], np.array([x['action']])), axis=1)
    states = np.stack(states)
    curr_len = len(states)
    if curr_len <= seq_len:
        pad_len = seq_len - curr_len
        states = np.pad(states, pad_width=[(pad_len, 0),(0, 0)], mode='constant', constant_values=0.)
    else:
        states = states[-seq_len:, :]

    all_states.append(states)

    label.append(df.iloc[-1]['reward'])

label = np.array([1 if r == 100. else 0 for r in label])
states = np.stack(all_states)

In [ ]:
from sklearn.model_selection import train_test_split

train_id, test_id = train_test_split(range(len(label)), test_size=.2, stratify=label)

X_train = states[train_id]
y_train = label[train_id]

y_test = label[test_id]
valid_id, test_id = train_test_split(test_id, test_size=.5, stratify=y_test)
X_valid = states[valid_id]
y_valid = label[valid_id]

X_test = states[test_id]
y_test = label[test_id]


In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(X_train).type(torch.FloatTensor), torch.from_numpy(y_train).type(torch.LongTensor))
test_data = TensorDataset(torch.from_numpy(X_test).type(torch.FloatTensor), torch.from_numpy(y_test).type(torch.LongTensor))
valid_data = TensorDataset(torch.from_numpy(X_valid).type(torch.FloatTensor), torch.from_numpy(y_valid).type(torch.LongTensor))


# dataloaders
batch_size = 20
# make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)

In [ ]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()
print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

In [ ]:
import torch.nn as nn

class LSTMValidator(nn.Module):
    """
    The RNN model that will be used to perform prediction on high/low learning gain.
    """

    def __init__(self, output_size, seq_len, state_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super().__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        
        self.lstm = nn.LSTM(state_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        lstm_out, hidden = self.lstm(x, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

In [ ]:
# Instantiate the model w/ hyperparams
output_size = 1
seq_len = 700
hidden_dim = 256
n_layers = 2
net = LSTMValidator(output_size, seq_len, state_dim, hidden_dim, n_layers, drop_prob=0.5)
print(net)

In [ ]:
import pdb

# loss and optimization functions
lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)


# training params

epochs = 4

counter = 0
print_every = 1
clip=5 # gradient clipping

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        if len(inputs) < batch_size:
            print('skipping {0} inputs in train'.format(len(inputs)))
            continue
        counter += 1

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
#         inputs = inputs.type(torch.LongTensor)
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:
                if len(inputs) < batch_size:
                    print('skipping {0} inputs in valid'.format(len(inputs)))
                    continue

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

In [ ]:
# Get test data loss and accuracy

test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])
    
    # get predicted outputs
    output, h = net(inputs, h)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) 
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

# random

In [12]:
import pandas as pd

df_org = pd.read_pickle('../processed_data/narrative_trajectories.pkl')
df_sim = pd.read_pickle('../simulated_data/seed_0_sim_narr.pkl')
len(df_org), len(df_sim)

(3038, 7496)

In [13]:
len(df_org['student_id'].unique()), len(df_sim['student_id'].unique())

(398, 1000)

In [6]:
df.columns

Index(['student_id', 'step', 's_use_pda', 's_drop_obj', 's_test_obj',
       's_take_note', 's_end_game', 's_pickup_obj', 's_quiz',
       's_bryce_computer', 's_view_poster', 's_label_slide', 's_dialog_turn',
       's_worksheet', 's_read_book', 's_open_door', 's_talk_others',
       's_talk_bryce', 's_talk_ford_quen_rob', 's_talk_teresa',
       's_worksheet_submitted', 's_aes_bryce_symptoms',
       's_aes_teresa_symptoms', 's_aes_knowledge_quiz',
       's_aes_diagnosis_feedback', 's_aes_trigger_bryce_symptoms',
       's_aes_trigger_teresa_symptoms', 's_aes_trigger_knowledge_quiz',
       's_aes_trigger_diagnosis_feedback', 'action', 'action_name', 'done',
       's_static_gender', 's_static_game_freq', 's_static_pretest', 'reward',
       'state'],
      dtype='object')